In [25]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

In [ ]:
### TODO: MIGRAR PARA O FTPLIB


In [33]:
#https://docs.python.org/3/library/ftplib.html
#required brazilian ip
from ftplib import FTP

ftp = FTP('ftp.mtps.gov.br')
ftp.login()
ftp.encoding = 'latin1'
ftp.cwd('pdet/microdados/NOVO CAGED/Movimentações')
ftp.nlst()

['2020', '2021', 'Layout Novo Caged Movimentação.xlsx']

In [ ]:
#test option
ftp = FTP('ftp.dlptest.com')
ftp.login(user='dlpuser', passwd='rNrKYTX9g7z3RgJRmxWuGHbeu')
ftp.nlst()

In [2]:
print("Disponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.Disponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.\nDisponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.\n")

Disponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.Disponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.
Disponibilização de informações da execução orçamentária e financeira dos órgãos e entidades da Administração Direta e Indireta do Estado a partir de 2010, com detalhamento das despesas empenhadas, liquidadas e pagas, do ano e de restos a pagar, incluídas as intra-orçamentárias.



### Get Urls

In [6]:
download_dict = caged_novo.get_download_links()

estabelecimentos


URLError: <urlopen error ftp error: error_perm('550 The system cannot find the file specified. ')>

In [14]:
download_dict['movimentacoes']['check_download'].keys()

dict_keys(['202001', '202002', '202003'])

In [17]:
download_dict['movimentacoes']['must_download']

['ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202004.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202005.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202006.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202007.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202008.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202009.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202010.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202011.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202012.7z',
 'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Abril/CAGEDMOV202101.7z',
 'ftp://ftp.mtps.gov

### Download Data

In [3]:

import shutil
import urllib.request as request
from contextlib import closing

def download_data(download_link, download_path, filename):
    ## download do arquivo
    with closing(request.urlopen(download_link)) as r:
        with open(os.path.join(download_path, filename), "wb") as f:
            shutil.copyfileobj(r, f)

In [4]:
download_data(
    'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/Movimentações/2021/Março/CAGEDMOV202103.7z', 
    '.', 
    'CAGEDMOV202103.7z')